# Comandos y funciones útiles 

#### Cambiar el ancho del panel

In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

####  Exportación sin código

In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
print(nb_name)
!jupyter nbconvert --output-dir='/home/mato/Documentos/Datos/EDA/CEO' --no-input --to html $nb_name

In [ ]:
# Ocultando tags = 'no'
!jupyter nbconvert --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='{"no"}' --output-dir='/home/mato/Documentos/Datos/EDA/TasasDeUso/export' --no-input --to html $nb_name 

#### Exportar a HTML sin codigo y sin celdas con tag "no"

In [ ]:
# (desde la consola)
jupyter nbconvert archivo.ipynb --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='{"no"}' --no-input --to html

#### Exportar a HTML con gráficos interactivos

https://community.plotly.com/t/export-figurewidgets-from-jupyter-notebook-to-html/15556/2

- Run the notebook using the classic JupyterNotebook (not JupyterLab)
- Save notebook using the “Widgets -> Save Notebook Widgets State” menu item
- Use nbconvert on the command line to convert the notebook to HTML (See https://github.com/jupyter/nbconvert 169). I have nbconvert version 5.4.0 installed.

    $ jupyter nbconvert --to html mynotebook.ipynb


#### Jupyterthemes

In [4]:
# !pip install jupyterthemes 
!jt -t grade3 -tf firacode -tfs 13 -mathfs 110 -fs 13 -N -T

#### Buscar parte de una cadena, de una lista, en otra lista

In [ ]:
# https://stackoverflow.com/questions/4843158/check-if-a-python-list-item-contains-a-string-inside-another-string
lista = ['esto es argentina',
 'esto es francia y chile',
 'esto es nada',
 'esto es francia',
 'esto es chile',
 'nada']
cadenas = ['argentina', 'francia', 'chile', 'brasil']

matching = [s for s in lista if any(xs in s for xs in cadenas)]
matching

#### Formatear números.

In [ ]:
pd.set_option('float_format', '{:f}'.format)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# https://re-thought.com/how-to-suppress-scientific-notation-in-pandas/
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.reset_option('display.float_format')

#### Reemplazamos infinito y NaN por 0

In [ ]:
cuadro03 = cuadro03.replace([np.inf, -np.inf, np.nan], 0).sort_values(by=['OpSobreDeb100'], ascending = False)

#### Renombramos columnas

In [ ]:
cuadro03.rename(columns = {'OP': 'Suma de OP', 
                           'NemesisDeb': 'Suma de Debitos', 
                           'OpSobreDeb100':'OP sobre Debitos (por 100)'}, 
                inplace = True)

#### Concatenar columnas 

In [ ]:
df["period"] = df["Year"].astype(str) + df["quarter"]

#### Generamos dataframes ordenados por diferentes columnas 

In [ ]:
columns = [
    'Suma de NemesisPrecarga', 'NemesisLiq', 'NemesisDeb', 'PreOP', 'OP',
    'Pagado', 'Anticipos / NC', 'Deuda'
]

list_of_dfs = {}
for column in columns:
    list_of_dfs[column] = df.sort_values(by=[column],
                                         ascending=False).iloc[0:20, :]

# Mostramos una de las resultantes. Sería un vector de DF, cada uno con el nombre de la columna como id
list_of_dfs['NemesisLiq']

#### Guardamos DF en excel, con varias solapas 

In [ ]:
writer = pd.ExcelWriter('/home/mato/Documentos/Datos/EDA/liquidaciones/AnalisisData.xlsx', engine='xlsxwriter')

cuadro01.to_excel(writer, sheet_name='PPXop', index= False)
cuadro02.to_excel(writer, sheet_name='PPXdebito', index= False)
cuadro03.to_excel(writer, sheet_name='MayordebitoxOP', index= False)
cuadro04.to_excel(writer, sheet_name='cocientes', index= False)
cuadro05.to_excel(writer, sheet_name='AltaFact_bajoDeb', index= False)
cuadro06.to_excel(writer, sheet_name='BajaFact_bajoDeb', index= False)
cuadro07.to_excel(writer, sheet_name='OPsRepetidas')

writer.save()

#### Cantidad de filas y columnas para mostrar

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

####  Reemplzamos NaN

In [ ]:
df.fillna(0) #  en todo el DF

#### Bins

In [ ]:
bins = [-np.inf, 0, 10, 20,30,40,50,60,70,80,90,100,150,250,500,1000, np.inf]
labels = ['bajo','0', '20','40'] # Las etiquetas tienen que ser una menos que los bins
df['range'] = pd.cut(df['Diff_deuda_Porcentaje'], labels = labels, bins=bins, include_lowest=True)
df.groupby('range')['Diff_deuda_Porcentaje'].count()

####  Melt DF

In [ ]:
fechas_gra = df.groupby('Año-Mes').agg({'OP':'sum','NemesisDeb':'sum','Pagado':'sum','Anticipos / NC':'sum','Deuda':'sum'}).sort_values(by = 'Año-Mes').reset_index()
fechas_gra = fechas_gra.melt(id_vars=['Año-Mes'], value_vars=['OP','Pagado','Deuda'],  var_name='Concepto', value_name='Pesos')

#### ALTAIR - Bucle y descarga de imágenes

In [ ]:
import altair as alt
alt.renderers.enable('notebook')
alt.renderers.enable('default')
from altair_saver import save
import selenium.webdriver
driver = selenium.webdriver.Chrome()

g = cuadro04.head(30)
# g
var = ['PagadoSobreOP','DeudaSobreOp','DebitoSobreOp','DebitosSobrePagado']
import os
for var in var:
    chart = alt.Chart(g).mark_bar(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        alt.X("Denominacion", axis=alt.Axis(title='Prestador/Proveedor')),
        alt.Y(var, axis= alt.Axis(title='%'))
    ).properties(
        title=var,
        width=800,
        height=300)
    print(chart)
    chart.display()
    cwd = os.getcwd()
    path = cwd + '/graficos/' + var + '.png'
    save(chart, path)

#### Histogramas 

In [ ]:
import pylab as pl

cuadro05['DebitoSobreOp'].hist(bins=50, range=[0, 25])
pl.title("Debitos sobre OP - TOP 30")
pl.xlabel("%")
pl.ylabel("Cantidad")

#### Spatial Join

In [ ]:
### https://towardsdatascience.com/how-to-easily-join-data-by-location-in-python-spatial-join-197490ff3544

# 1- Puntos
df = pd.read_csv('/home/mato/Documentos/Datos/EDA/Walter/prestadores_CB3_DATA_STUDIO_QGIS_IV.csv')

# 2 - Conversión a Geopandas Geodataframe
gdf_df = gpd.GeoDataFrame(df,   
                          geometry=gpd.points_from_xy(df.DirGeoX, df.DirGeoY))

# 3 - Capa de barrios
barrios = gpd.read_file('/home/mato/Documentos/Datos/EDA/mapas/barriosCABA/barrios_badata.shp')

# barrios.crs = barrios.to_crs("EPSG:4326")
# gdf_df.crs = gdf_df.to_crs("epsg:3395") #Mercator-projection

gdf_df.crs = "EPSG:4326"
barrios.crs = 'EPSG:4326'

# Hacemos el spatial join
sjoined_df = gpd.sjoin(gdf_df, barrios, op='within')

#### Limites para los mapas

In [ ]:
# provincias es el nombre del shape al cual queremos ajustarnos.
ax.set_ylim(provincias.bounds.miny.min(), provincias.bounds.maxy.max())
ax.set_xlim(provincias.bounds.minx.min(), provincias.bounds.maxx.max())

#### Reemplazar varios valores en una columna 

In [ ]:
df3['month'] = df3['month'].replace(['enero','febrero','marzo','abril','mayo','junio','julio','agosto',
                                     'septiembre','octubre','noviembre','diciembre'],
                                     [1,2,3,4,5,6,7,8,9,10,11,12])

#### Fechas desde columnas con dia, mes y año 

In [ ]:
df3 = df2[["FechaServicioDetalle_año_aut", "FechaServicioDetalle_mes_aut", "FechaServicioDetalle_dia_aut"]].copy()
df3.columns = ["year", "month", "day"]
df3['month'] = df3['month'].str.strip()
df3['year'] = df3['year'].str.strip()
df3['day'] = df3['day'].str.strip()
df3['month'] = df3['month'].replace(['enero','febrero','marzo','abril','mayo','junio','julio','agosto',
                                     'septiembre','octubre','noviembre','diciembre'],
                                     [1,2,3,4,5,6,7,8,9,10,11,12])

df3['fecha'] = pd.to_datetime(df3[['year', 'month', 'day']])

#### Agrupado contar valores sin NaN

In [ ]:
df.groupby('columna').agg(
    **{
        'Monto No Nulo': ('Monto_suma', lambda x: x.count()),
    })

#### Agrupado con la misma columna

In [ ]:
animals.groupby("kind").agg(
                            min_height=pd.NamedAgg(column='height', aggfunc='min'),
                            max_height=pd.NamedAgg(column='height', aggfunc='max'),
                            average_weight=pd.NamedAgg(column='weight', aggfunc=np.mean),
                            )

df2.groupby(['provincia_pres']).agg(**{
                                    'Monto Promedio': ('Monto_suma', 'mean'),
                                    'Monto Total Sumado': ('Monto_suma', 'max')
                                    })
    

df2.groupby(['id_afiliado_aut']).\
                     agg(**{
                         'Gasto Promedio Por Afiliado':pd.NamedAgg(column='Monto_suma', aggfunc='min'),
                         'Monto Total Afiliado':pd.NamedAgg(column='Monto_suma', aggfunc='sum'),
                         })

#### Colorear celdas según valores

In [ ]:
def pintura(val):
    global color1
    if val < promedio:
        color1 = 'green'
        print('a')
    elif val > promedio:
        color1 = 'yellow'
    return 'background-color: %s' % color1

prom_aut.style.applymap(pintura, subset=['Monto Promedio por Aut'])